# Project COGS 189: EEG-Based Attention State Classification Using Machine Learning


Add your names & PIDs here

Xavier Lopez A17068366

#### Recommended Environment Setup
```  conda create --name "cogs189" python=3.9   ```

``` conda activate cogs189 ```

#### Now run the following cell to install the requirements.txt

In [3]:
# Install necessary libraries (run only once per session)
import os
if not os.path.exists("requirements_installed.txt"):
    import sys
    !{sys.executable} -m pip install -r requirements.txt --quiet
    with open("requirements_installed.txt", "w") as f:
        f.write("installed")

#### Test to see if the installation worked by importing the needed libraries

In [1]:
import zipfile
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import welch, butter, lfilter
import mne  # EEG processing library

#### Proceeding to handling the data

In [12]:
zipped_name = "data_preprocessed_python.zip"
unzipped_folder = "data_preprocessed_python"

if not os.path.exists(unzipped_folder):
    print("Extracting files...")
    with zipfile.ZipFile(zipped_name, 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())
    print(f"Extracted files into: {os.getcwd()}")
else:
    print("Files already extracted.")

files = [f for f in os.listdir(unzipped_folder) if f.endswith(".dat")]
print(f"Found {len(files)} EEG data files in {unzipped_folder}.")

Extracting files...
Extracted files into: /Users/xavier/Library/CloudStorage/GoogleDrive-xlopez@ucsd.edu/My Drive/4th Year/Winter Quarter/COGS189
Found 32 EEG data files in data_preprocessed_python.
